In [1]:
import os
import glob
import json
import copy
import numpy as np

In [2]:
this_path = './paths'

buttons = ['auto_help', 'clinic_get_cost', 'clinic_free_cost_calc', 'clinics_get_clinic_in_country', 'callback', 'clinic_get_cost_header', 'clinic_diagnostic', 'article_from_text', 'second_opinion', 'clinics_get_country', 'doctor_consult_price', 'prices_in_country', 'unknown', 'banner_get_best_clinic', 'wanna_cheaper', 'search_get_cost', 'doctor_diagnostic', 'sign_in', 'doctor_get_consult_header', 'clinic_operation', 'get_consult', 'program_from_text', 'doctor_get_consult', 'identify', 'Requested Callback', 'Requested Treatment']

In [3]:
files = [os.path.basename(x) for x in glob.glob('%s/*.json'%this_path)]
files = [os.path.basename(x) for x in glob.glob('%s/*, session_1.json'%this_path)]

In [4]:
all_paths=[]
for file_name in files:
    with open('%s/%s'%(this_path,file_name)) as f:
        all_paths += f.readlines()

In [34]:
original_all_paths =[]
for line in all_paths:
    #if 'clinic_get_cost ,' in line:
    #    original_all_paths += [line]
    #elif 'clinic_get_cost\n' in line:
    #    original_all_paths += [line]
    if 'Requested Treatment' in line:
        original_all_paths += [line]

In [8]:
original_all_paths = copy.deepcopy(all_paths)

In [22]:
def get_chunks(path_list, pos):
    num_step = []
    for path in path_list:
        try:
            chunk = path.strip().split(', ')[pos].strip()
            if chunk == '':
                num_step += ['/']
            else:
                num_step += [chunk]
        except:
            pass    
    return num_step

In [23]:
def get_chunks_set_count(chunks):
    chunks_set = list(set(chunks))
    chunks_set_count = [ [chunks.count(i), i, []] for i in chunks_set]
    return sorted(chunks_set_count, reverse = True)    

In [24]:
def filter_chunk_set_count(chunks_set_count):
    num_list = map(lambda x: x[0], chunks_set_count)
    max_value = max(num_list)
    min_value = min(num_list)
    mean_value = np.mean(num_list)
    if (max_value - min_value) > mean_value:
        return filter(lambda x: x[0] > mean_value, chunks_set_count)
    else: return chunks_set_count

In [25]:
def get_paths_by(chunk_list, original_paths):
    all_paths = []
    for i in original_paths:
        try:
            for k, x in enumerate(chunk_list):
                if x != i.strip().split(', ')[k].strip():
                    raise
            all_paths += [i]
        except:
            pass
    return all_paths

In [ ]:
init_list = []
init_chunk_list = []
init_list += filter_chunk_set_count(get_chunks_set_count(get_chunks(get_paths_by(init_chunk_list, original_all_paths), 0, )))
for el in init_list:
    chunk_list = init_chunk_list + [el[1]]
    el[-1] += filter_chunk_set_count(get_chunks_set_count(get_chunks(get_paths_by(chunk_list, original_all_paths), 1, )))
    for el_2 in el[-1]:
        chunk_list_2 = chunk_list + [el_2[1]]
        el_2[-1] += filter_chunk_set_count(get_chunks_set_count(get_chunks(get_paths_by(chunk_list_2, original_all_paths), 2, )))
        

In [26]:
def explore(tree_list, tree_chunk_list, pos_num):
    #if pos_num > 2:
    #    return []
    for el in tree_list:
        try:
            chunk_list = tree_chunk_list + [el[1]]
            paths = get_paths_by(chunk_list, original_all_paths)
            el[-1] += filter_chunk_set_count(get_chunks_set_count(get_chunks(paths, pos_num)))
            explore(el[-1], chunk_list, pos_num+1)
        except:
            return []    

In [27]:
def print_list(f, tree_list, num_t):
    for i in tree_list:
        #print "\t" * num_t, i[0], i[1]
        f.write("\t" * num_t +str(i[0]) + ' ' + i[1] + '\n')
        if i[-1] != []:
            print_list(f, i[-1], num_t+1)

In [35]:
init_list = []
init_chunk_list = []
init_list += filter_chunk_set_count(get_chunks_set_count(get_chunks(get_paths_by(init_chunk_list, original_all_paths), 0, )))
explore(init_list, init_chunk_list, 1)

In [36]:
f = open('tree_paths_event_2.txt', 'w+')
print_list(f, init_list,0)
f.close()

In [30]:
a = get_chunks_set_count(get_chunks(get_paths_by(init_chunk_list, original_all_paths), 1, ))

In [33]:
for i in a:
    print i[0]/53506.0, i[1]

0.525604605091 /clinics/country/illness/
0.180297536725 /clinics/country/
0.128882742122 /clinic/clinic_name/
0.0712069674429 /clinics/illness/
0.0402384779277 /clinics/country/direction/
0.0159234478376 /clinics/country/city/
0.0126340971106 /
0.0100175681232 /article/article_name/
0.00252308152357 /clinics/direction/
0.00177550181288 /clinics/
0.00175681232011 /clinics/procedure/
0.00143909094307 /doctor/doctor_name/
0.000953164131125 /review/num/
0.000841027174522 /clientLK/
0.000635442754084 /clinics/operation/
0.000616753261316 /clinics/country/procedure/
0.000579374275782 /treatment/country/
0.000579374275782 /agency/agency_name/
0.000355100362576 /diagnosticsKind/num/
0.000336410869809 clinic_get_cost
0.000242963405973 /doc/doc_name/
0.000186894927672 /illnesses/
0.000186894927672 /doctors/direction/
0.000168205434904 /countries/
0.000149515942137 /diagnostic/num/
0.000149515942137 /about/
0.00013082644937 /clientLK/profile/
9.34474638358e-05 google
9.34474638358e-05 /clientLK/r